## Description

Queries coin gecko to see tokens between x and y market cap and then provide a list of them that have USDT margined perps listed on Binance

In [1]:
# Import required libraries
import requests
import pandas as pd
import time
from typing import List, Dict, Optional
import json

In [2]:
# Configuration parameters
MIN_MARKET_CAP = 50_000_000
MAX_MARKET_CAP = 60_000_000
RESULTS_PER_PAGE = 250  # CoinGecko API limit

print(f"Looking for tokens with market cap between ${MIN_MARKET_CAP:,} and ${MAX_MARKET_CAP:,}")
print(f"That's between ${MIN_MARKET_CAP/1_000_000:.0f}M and ${MAX_MARKET_CAP/1_000_000:.1f}M")

Looking for tokens with market cap between $50,000,000 and $60,000,000
That's between $50M and $60.0M


In [3]:
def get_coingecko_tokens_by_market_cap(min_cap: float, max_cap: float) -> List[Dict]:
    """
    Fetch cryptocurrency tokens from CoinGecko within specified market cap range.
    
    Args:
        min_cap: Minimum market cap in USD
        max_cap: Maximum market cap in USD
    
    Returns:
        List of token dictionaries with id, symbol, name, and market_cap
    """
    all_tokens = []
    page = 1
    
    while True:
        url = "https://api.coingecko.com/api/v3/coins/markets"
        params = {
            'vs_currency': 'usd',
            'order': 'market_cap_desc',
            'per_page': RESULTS_PER_PAGE,
            'page': page,
            'sparkline': 'false',
            'price_change_percentage': '24h'
        }
        
        try:
            print(f"Fetching page {page}...")
            response = requests.get(url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if not data:  # No more data
                break
                
            # Filter tokens by market cap range
            filtered_tokens = []
            for token in data:
                market_cap = token.get('market_cap', 0)
                if market_cap and min_cap <= market_cap <= max_cap:
                    filtered_tokens.append({
                        'id': token['id'],
                        'symbol': token['symbol'].upper(),
                        'name': token['name'],
                        'market_cap': market_cap,
                        'price': token.get('current_price', 0),
                        'market_cap_rank': token.get('market_cap_rank', 0)
                    })
            
            all_tokens.extend(filtered_tokens)
            
            # Check if we've gone past our max market cap (since results are ordered by market cap desc)
            min_market_cap_on_page = min([t.get('market_cap', float('inf')) for t in data if t.get('market_cap')])
            if min_market_cap_on_page < min_cap:
                print(f"Reached tokens below minimum market cap on page {page}")
                break
                
            page += 1
            time.sleep(15) 
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from CoinGecko: {e}")
            break
    
    print(f"Found {len(all_tokens)} tokens within market cap range")
    return all_tokens

In [4]:
def get_binance_usdt_perpetual_symbols() -> List[str]:
    """
    Fetch all USDT margined perpetual contract symbols from Binance.
    
    Returns:
        List of symbol strings (e.g., ['BTCUSDT', 'ETHUSDT', ...])
    """
    try:
        url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        # Extract symbols for USDT margined perpetual contracts
        usdt_perp_symbols = []
        for symbol_info in data.get('symbols', []):
            symbol = symbol_info.get('symbol', '')
            if (symbol.endswith('USDT') and 
                symbol_info.get('status') == 'TRADING' and
                symbol_info.get('contractType') == 'PERPETUAL'):
                usdt_perp_symbols.append(symbol)
        
        print(f"Found {len(usdt_perp_symbols)} USDT perpetual contracts on Binance")
        return sorted(usdt_perp_symbols)
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching Binance data: {e}")
        return []

In [5]:
def match_tokens_with_binance_perps(coingecko_tokens: List[Dict], binance_symbols: List[str]) -> List[Dict]:
    """
    Match CoinGecko tokens with Binance USDT perpetual contracts.
    
    Args:
        coingecko_tokens: List of token data from CoinGecko
        binance_symbols: List of Binance USDT perpetual symbols
    
    Returns:
        List of matched tokens with Binance symbol information
    """
    matched_tokens = []
    
    # Create a set of base symbols from Binance (remove USDT suffix)
    binance_base_symbols = {symbol.replace('USDT', '') for symbol in binance_symbols if symbol.endswith('USDT')}
    
    for token in coingecko_tokens:
        token_symbol = token['symbol'].upper()
        
        # Check if token symbol matches a Binance perpetual contract
        if token_symbol in binance_base_symbols:
            binance_symbol = f"{token_symbol}USDT"
            matched_token = token.copy()
            matched_token['binance_symbol'] = binance_symbol
            matched_tokens.append(matched_token)
    
    print(f"Matched {len(matched_tokens)} tokens with Binance USDT perpetual contracts")
    return matched_tokens

## Execute the Analysis

Now let's run the analysis to find tokens within our specified market cap range that have USDT perpetual contracts on Binance.

In [6]:
# Step 1: Get tokens from CoinGecko within market cap range
print("=== Fetching tokens from CoinGecko ===")
coingecko_tokens = get_coingecko_tokens_by_market_cap(MIN_MARKET_CAP, MAX_MARKET_CAP)

if coingecko_tokens:
    print(f"\nTop 10 tokens by market cap:")
    for i, token in enumerate(sorted(coingecko_tokens, key=lambda x: x['market_cap'], reverse=True)[:10]):
        print(f"{i+1:2d}. {token['symbol']:8s} - {token['name']:25s} - ${token['market_cap']:>12,.0f}")

    print(f"Bottom 10 tokens by market cap:")
    for i, token in enumerate(sorted(coingecko_tokens, key=lambda x: x['market_cap'])[:10]):
        print(f"{i+1:2d}. {token['symbol']:8s} - {token['name']:25s} - ${token['market_cap']:>12,.0f}")
else:
    print("No tokens found in the specified market cap range")

=== Fetching tokens from CoinGecko ===
Fetching page 1...
Fetching page 2...
Fetching page 3...
Reached tokens below minimum market cap on page 3
Found 76 tokens within market cap range

Top 10 tokens by market cap:
 1. SCRVUSD  - Savings crvUSD            - $  59,973,056
 2. MVL      - MVL                       - $  59,956,347
 3. 1R0R     - R0AR Token                - $  59,724,470
 4. PALMO    - ORCIB                     - $  59,704,881
 5. CSUSDL   - Coinshift USDL Morpho Vault - $  59,550,575
 6. ZEDXION  - Zedxion                   - $  59,503,536
 7. WHITE    - Whiteheart                - $  59,480,022
 8. SSOL     - Solayer Staked SOL        - $  59,460,712
 9. RLC      - iExec RLC                 - $  59,427,878
10. XUSD     - StraitsX XUSD             - $  59,420,938
Bottom 10 tokens by market cap:
 1. VBUSDC   - VaultBridge Bridged USDC (Katana) - $  50,036,986
 2. MANTA    - Manta Network             - $  50,055,283
 3. GOAT     - Goatseus Maximus          - $  50,101,230
 

In [7]:
# Step 2: Get Binance USDT perpetual symbols
print("\n=== Fetching Binance USDT perpetual contracts ===")
binance_symbols = get_binance_usdt_perpetual_symbols()

if binance_symbols:
    print(f"\nFirst 20 Binance USDT perpetual symbols:")
    for i, symbol in enumerate(binance_symbols[:20]):
        print(f"{symbol}", end="  ")
        if (i + 1) % 5 == 0:  # New line every 5 symbols
            print()
    if len(binance_symbols) > 20:
        print(f"\n... and {len(binance_symbols) - 20} more")
else:
    print("No Binance USDT perpetual contracts found")


=== Fetching Binance USDT perpetual contracts ===
Found 518 USDT perpetual contracts on Binance

First 20 Binance USDT perpetual symbols:
0GUSDT  1000000BOBUSDT  1000000MOGUSDT  1000BONKUSDT  1000CATUSDT  
1000CHEEMSUSDT  1000FLOKIUSDT  1000LUNCUSDT  1000PEPEUSDT  1000RATSUSDT  
1000SATSUSDT  1000SHIBUSDT  1000WHYUSDT  1000XECUSDT  1000XUSDT  
1INCHUSDT  1MBABYDOGEUSDT  2ZUSDT  4USDT  A2ZUSDT  

... and 498 more


In [8]:
# Step 3: Match tokens with Binance perpetual contracts
print("\n=== Matching tokens with Binance perpetual contracts ===")
if coingecko_tokens and binance_symbols:
    matched_tokens = match_tokens_with_binance_perps(coingecko_tokens, binance_symbols)
    
    if matched_tokens:
        print(f"\n✅ Found {len(matched_tokens)} tokens that meet all criteria:")
        print(f"{'Rank':<4} {'Symbol':<8} {'Binance':<12} {'Name':<25} {'Market Cap':<15} {'Price'}")
        print("-" * 80)
        
        # Sort by market cap descending
        sorted_tokens = sorted(matched_tokens, key=lambda x: x['market_cap'], reverse=True)
        
        for i, token in enumerate(sorted_tokens):
            rank = token.get('market_cap_rank', 'N/A')
            symbol = token['symbol']
            binance_symbol = token['binance_symbol']
            name = token['name'][:24]  # Truncate long names
            market_cap = f"${token['market_cap']:,.0f}"
            price = f"${token['price']:.4f}" if token['price'] > 0 else "N/A"
            
            print(f"{rank:<4} {symbol:<8} {binance_symbol:<12} {name:<25} {market_cap:<15} {price}")
    else:
        print("❌ No tokens found that match both criteria")
else:
    print("❌ Cannot proceed - missing data from previous steps")


=== Matching tokens with Binance perpetual contracts ===
Matched 29 tokens with Binance USDT perpetual contracts

✅ Found 29 tokens that meet all criteria:
Rank Symbol   Binance      Name                      Market Cap      Price
--------------------------------------------------------------------------------
668  RLC      RLCUSDT      iExec RLC                 $59,427,878     $0.8210
675  BANK     BANKUSDT     Lorenzo Protocol          $59,225,882     $0.1391
678  METIS    METISUSDT    Metis                     $58,937,982     $8.9500
679  HEMI     HEMIUSDT     Hemi                      $58,820,578     $0.0602
682  PEOPLE   PEOPLEUSDT   ConstitutionDAO           $57,788,240     $0.0114
683  VVV      VVVUSDT      Venice Token              $57,615,721     $1.4500
686  AEVO     AEVOUSDT     Aevo                      $57,279,044     $0.0627
688  SPELL    SPELLUSDT    Spell                     $56,996,316     $0.0003
689  LAYER    LAYERUSDT    Solayer                   $56,836,248     $0

In [9]:
# Step 4: Create DataFrame and save results
if 'matched_tokens' in locals() and matched_tokens:
    print("\n=== Creating DataFrame and saving results ===")
    
    # Create DataFrame
    df = pd.DataFrame(matched_tokens)
    df = df.sort_values('market_cap', ascending=False).reset_index(drop=True)
    
    # Add some additional calculated columns
    df['market_cap_billions'] = df['market_cap'] / 1_000_000_000
    df['market_cap_millions'] = df['market_cap'] / 1_000_000
    
    print(f"Created DataFrame with {len(df)} tokens")
    print(f"\nDataFrame columns: {list(df.columns)}")
    
    # Display the DataFrame
    print(f"\nFinal results:")
    display(df[['market_cap_rank', 'symbol', 'name', 'binance_symbol', 'market_cap_billions', 'price']])
    
    # Save to CSV
    output_file = '../data/filtered_tokens.csv'
    df.to_csv(output_file, index=False)
    print(f"\n💾 Results saved to: {output_file}")
    
    # Summary statistics
    print(f"\n📊 Summary Statistics:")
    print(f"   • Total tokens found: {len(df)}")
    print(f"   • Average market cap: ${df['market_cap'].mean():,.0f}")
    print(f"   • Median market cap: ${df['market_cap'].median():,.0f}")
    print(f"   • Market cap range: ${df['market_cap'].min():,.0f} - ${df['market_cap'].max():,.0f}")
    print(f"   • Average price: ${df['price'].mean():.4f}")
    
else:
    print("❌ No matched tokens to save")


=== Creating DataFrame and saving results ===
Created DataFrame with 29 tokens

DataFrame columns: ['id', 'symbol', 'name', 'market_cap', 'price', 'market_cap_rank', 'binance_symbol', 'market_cap_billions', 'market_cap_millions']

Final results:


,market_cap_rank,symbol,name,binance_symbol,market_cap_billions,price
0,668,RLC,iExec RLC,RLCUSDT,0.059428,0.821026
1,675,BANK,Lorenzo Protocol,BANKUSDT,0.059226,0.139114
2,678,METIS,Metis,METISUSDT,0.058938,8.950000
3,679,HEMI,Hemi,HEMIUSDT,0.058821,0.060192
4,682,PEOPLE,ConstitutionDAO,PEOPLEUSDT,0.057788,0.011369
5,683,VVV,Venice Token,VVVUSDT,0.057616,1.450000
6,686,AEVO,Aevo,AEVOUSDT,0.057279,0.062673
7,688,SPELL,Spell,SPELLUSDT,0.056996,0.000336
8,689,LAYER,Solayer,LAYERUSDT,0.056836,0.270728
9,695,CHR,Chromia,CHRUSDT,0.056164,0.066269



💾 Results saved to: ../data/filtered_tokens.csv

📊 Summary Statistics:
   • Total tokens found: 29
   • Average market cap: $54,412,972
   • Median market cap: $55,215,393
   • Market cap range: $50,055,283 - $59,427,878
   • Average price: $0.5549


## Additional Analysis (Optional)

You can modify the parameters above to explore different market cap ranges or add additional filtering criteria.

In [10]:
# Optional: Quick analysis of different market cap ranges
market_cap_ranges = [
    (10_000_000, 50_000_000, "10M-50M"),
    (50_000_000, 200_000_000, "50M-200M"),
    (200_000_000, 1_000_000_000, "200M-1B"),
    (1_000_000_000, 5_000_000_000, "1B-5B"),
    (5_000_000_000, 20_000_000_000, "5B-20B")
]

print("=== Quick analysis of different market cap ranges ===")
for min_cap, max_cap, label in market_cap_ranges:
    # Filter existing data instead of making new API calls
    if 'matched_tokens' in locals() and matched_tokens:
        filtered = [t for t in matched_tokens if min_cap <= t['market_cap'] <= max_cap]
        print(f"{label:>8}: {len(filtered):3d} tokens with Binance USDT perps")
    else:
        print(f"{label:>8}: No data available (run analysis above first)")

=== Quick analysis of different market cap ranges ===
 10M-50M:   0 tokens with Binance USDT perps
50M-200M:  29 tokens with Binance USDT perps
 200M-1B:   0 tokens with Binance USDT perps
   1B-5B:   0 tokens with Binance USDT perps
  5B-20B:   0 tokens with Binance USDT perps
